# Multi-Armed Bandit Algorithms: Mathematical Analysis

This notebook provides detailed mathematical derivations and ablation studies for the three bandit algorithms implemented in this project.

**Author**: Your Name  
**Date**: January 2026  
**Project**: News Recommendation System with MAB Algorithms

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Import our modules
from bandit_algorithms import EpsilonGreedy, UCB, ThompsonSampling
from simulation import NewsEnvironment, run_simulation, compare_algorithms

# Plotting setup
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Problem Formulation

### 1.1 The Multi-Armed Bandit Problem

We have $K$ arms (articles), each with unknown reward distribution. At each time step $t$:

1. Select arm $a_t \in \{1, 2, ..., K\}$
2. Observe reward $r_t \sim P(r|a_t)$
3. Update beliefs about arm rewards

**Goal**: Maximize cumulative reward $\sum_{t=1}^{T} r_t$

### 1.2 Regret Definition

Let $\mu^* = \max_a \mu_a$ be the optimal mean reward.

The **regret** at time $T$ is:

$$R_T = T\mu^* - \sum_{t=1}^{T} r_t$$

Equivalently:

$$R_T = \sum_{a=1}^{K} \Delta_a \mathbb{E}[N_a(T)]$$

where:
- $\Delta_a = \mu^* - \mu_a$ (suboptimality gap)
- $N_a(T)$ = number of times arm $a$ was pulled in $T$ rounds

## 2. Epsilon-Greedy Algorithm

### 2.1 Algorithm Description

At each time step $t$:

$$a_t = \begin{cases}
\text{random arm} & \text{with probability } \epsilon \\
\arg\max_a \hat{\mu}_a(t) & \text{with probability } 1-\epsilon
\end{cases}$$

where $\hat{\mu}_a(t) = \frac{1}{N_a(t)} \sum_{s=1}^{t} r_s \mathbb{1}(a_s = a)$

### 2.2 Regret Analysis

**Theorem**: For constant $\epsilon$, Epsilon-Greedy has **linear regret**:

$$\mathbb{E}[R_T] = \Omega(T)$$

**Proof sketch**:
- With probability $\epsilon$, we explore uniformly
- Expected pulls of suboptimal arm $a$: $\mathbb{E}[N_a(T)] \geq \frac{\epsilon T}{K}$
- Regret contribution: $\Delta_a \cdot \frac{\epsilon T}{K}$
- Total regret: $R_T \geq \frac{\epsilon T}{K} \sum_{a: \Delta_a > 0} \Delta_a = \Omega(T)$

### 2.3 Decaying Epsilon

To achieve sublinear regret, use $\epsilon_t = \frac{c}{t^\alpha}$ with $0 < \alpha < 1$:

$$\mathbb{E}[R_T] = O(T^{1-\alpha})$$

Common choice: $\epsilon_t = \min\{1, \frac{cK}{d^2 t}\}$ where $c > 0$, $d = \min_{a: \Delta_a > 0} \Delta_a$

In [ ]:
# Ablation Study 1: Effect of Epsilon on Performance

def epsilon_ablation_study(epsilon_values, n_rounds=1000, n_simulations=50):
    """
    Study the effect of different epsilon values on performance
    """
    env = NewsEnvironment(n_articles=10, seed=42)
    
    results = []
    
    for epsilon in epsilon_values:
        print(f"Testing epsilon={epsilon}...")
        
        total_rewards = []
        final_regrets = []
        exploration_ratios = []
        
        for _ in range(n_simulations):
            algo = EpsilonGreedy(n_arms=10, epsilon=epsilon)
            sim_results = run_simulation(algo, env, n_rounds)
            
            total_rewards.append(sum(sim_results['rewards']))
            final_regrets.append(sim_results['regret'][-1])
            exploration_ratios.append(algo.get_exploration_ratio())
        
        results.append({
            'epsilon': epsilon,
            'avg_reward': np.mean(total_rewards),
            'std_reward': np.std(total_rewards),
            'avg_regret': np.mean(final_regrets),
            'std_regret': np.std(final_regrets),
            'avg_exploration_ratio': np.mean(exploration_ratios)
        })
    
    return pd.DataFrame(results)

# Run ablation
epsilon_values = [0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5]
epsilon_results = epsilon_ablation_study(epsilon_values)

print(epsilon_results)

In [ ]:
# Visualize epsilon ablation

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Average reward vs epsilon
axes[0].errorbar(epsilon_results['epsilon'], epsilon_results['avg_reward'],
                yerr=epsilon_results['std_reward'], marker='o', capsize=5)
axes[0].set_xlabel('Epsilon (ε)', fontsize=12)
axes[0].set_ylabel('Average Total Reward', fontsize=12)
axes[0].set_title('Effect of Epsilon on Total Reward', fontsize=14)
axes[0].grid(True, alpha=0.3)

# Average regret vs epsilon
axes[1].errorbar(epsilon_results['epsilon'], epsilon_results['avg_regret'],
                yerr=epsilon_results['std_regret'], marker='o', capsize=5, color='red')
axes[1].set_xlabel('Epsilon (ε)', fontsize=12)
axes[1].set_ylabel('Average Final Regret', fontsize=12)
axes[1].set_title('Effect of Epsilon on Cumulative Regret', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find optimal epsilon
optimal_idx = epsilon_results['avg_regret'].idxmin()
optimal_epsilon = epsilon_results.loc[optimal_idx, 'epsilon']
print(f"\nOptimal epsilon: {optimal_epsilon}")
print(f"Average regret at optimal epsilon: {epsilon_results.loc[optimal_idx, 'avg_regret']:.2f}")

## 3. Upper Confidence Bound (UCB) Algorithm

### 3.1 Algorithm Description

UCB1 selects arm with highest upper confidence bound:

$$a_t = \arg\max_a \left[ \hat{\mu}_a(t) + \sqrt{\frac{2\ln t}{N_a(t)}} \right]$$

The second term is the **exploration bonus** based on:
- **Optimism in the face of uncertainty**: Assume uncertain arms might be best
- **Confidence radius**: Decreases as we learn more about each arm

### 3.2 Hoeffding's Inequality Foundation

UCB is based on Hoeffding's inequality:

$$P(|\hat{\mu}_a - \mu_a| \geq u) \leq 2e^{-2N_a u^2}$$

Setting $2e^{-2N_a u^2} = \frac{1}{t^4}$ and solving for $u$:

$$u = \sqrt{\frac{2\ln t + c}{N_a}}$$

This gives us confidence that $\mu_a \in [\hat{\mu}_a - u, \hat{\mu}_a + u]$ with high probability.

### 3.3 Regret Bound

**Theorem (Auer et al., 2002)**: UCB1 achieves **logarithmic regret**:

$$\mathbb{E}[R_T] \leq \sum_{a: \Delta_a > 0} \left( \frac{8\ln T}{\Delta_a} + \left(1 + \frac{\pi^2}{3}\right) \Delta_a \right)$$

This is asymptotically optimal!

**Key insight**: Regret grows as $O(\log T)$ rather than $O(T)$.

In [ ]:
# Ablation Study 2: Effect of UCB Confidence Parameter

def ucb_c_ablation_study(c_values, n_rounds=1000, n_simulations=50):
    """
    Study the effect of different c parameter values on UCB performance
    """
    env = NewsEnvironment(n_articles=10, seed=42)
    
    results = []
    
    for c in c_values:
        print(f"Testing c={c}...")
        
        total_rewards = []
        final_regrets = []
        
        for _ in range(n_simulations):
            algo = UCB(n_arms=10, c=c)
            sim_results = run_simulation(algo, env, n_rounds)
            
            total_rewards.append(sum(sim_results['rewards']))
            final_regrets.append(sim_results['regret'][-1])
        
        results.append({
            'c': c,
            'avg_reward': np.mean(total_rewards),
            'std_reward': np.std(total_rewards),
            'avg_regret': np.mean(final_regrets),
            'std_regret': np.std(final_regrets)
        })
    
    return pd.DataFrame(results)

# Run ablation
c_values = [0.5, 1.0, 1.414, 2.0, 3.0, 5.0]
ucb_results = ucb_c_ablation_study(c_values)

print(ucb_results)

In [ ]:
# Visualize UCB c parameter ablation

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Average reward vs c
axes[0].errorbar(ucb_results['c'], ucb_results['avg_reward'],
                yerr=ucb_results['std_reward'], marker='s', capsize=5, color='green')
axes[0].axvline(x=np.sqrt(2), color='red', linestyle='--', label='√2 (theoretical)')
axes[0].set_xlabel('Confidence Parameter (c)', fontsize=12)
axes[0].set_ylabel('Average Total Reward', fontsize=12)
axes[0].set_title('Effect of c on UCB Total Reward', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Average regret vs c
axes[1].errorbar(ucb_results['c'], ucb_results['avg_regret'],
                yerr=ucb_results['std_regret'], marker='s', capsize=5, color='orange')
axes[1].axvline(x=np.sqrt(2), color='red', linestyle='--', label='√2 (theoretical)')
axes[1].set_xlabel('Confidence Parameter (c)', fontsize=12)
axes[1].set_ylabel('Average Final Regret', fontsize=12)
axes[1].set_title('Effect of c on UCB Cumulative Regret', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Thompson Sampling

### 4.1 Bayesian Framework

Thompson Sampling takes a **Bayesian approach**:

1. Maintain posterior distribution over each arm's mean reward: $P(\mu_a | \text{data})$
2. Sample $\tilde{\mu}_a \sim P(\mu_a | \text{data})$ for each arm
3. Select $a_t = \arg\max_a \tilde{\mu}_a$

### 4.2 Beta-Bernoulli Conjugacy

For binary rewards (click/no-click), we use **Beta-Bernoulli conjugacy**:

**Prior**: $\mu_a \sim \text{Beta}(\alpha_0, \beta_0)$

**Likelihood**: $r \sim \text{Bernoulli}(\mu_a)$

**Posterior**: $\mu_a | \text{data} \sim \text{Beta}(\alpha_0 + S_a, \beta_0 + F_a)$

where:
- $S_a$ = number of successes (clicks) for arm $a$
- $F_a$ = number of failures (no clicks) for arm $a$

### 4.3 Beta Distribution Properties

The Beta distribution $\text{Beta}(\alpha, \beta)$ has:

**PDF**: $f(x; \alpha, \beta) = \frac{x^{\alpha-1}(1-x)^{\beta-1}}{B(\alpha, \beta)}$

**Mean**: $\mathbb{E}[X] = \frac{\alpha}{\alpha + \beta}$

**Variance**: $\text{Var}(X) = \frac{\alpha\beta}{(\alpha + \beta)^2(\alpha + \beta + 1)}$

**Mode**: $\text{mode}(X) = \frac{\alpha - 1}{\alpha + \beta - 2}$ (for $\alpha, \beta > 1$)

### 4.4 Regret Bound

**Theorem (Agrawal & Goyal, 2012)**: Thompson Sampling achieves:

$$\mathbb{E}[R_T] = O\left(\sum_{a: \Delta_a > 0} \frac{\ln T}{\Delta_a}\right)$$

This matches the lower bound up to constant factors!

In [ ]:
# Visualization: Thompson Sampling Beta Distributions Evolution

def visualize_thompson_evolution(n_rounds=100):
    """
    Visualize how Beta distributions evolve over time
    """
    env = NewsEnvironment(n_articles=5, seed=42)
    algo = ThompsonSampling(n_arms=5)
    
    # Snapshots at different time points
    snapshots = [10, 30, 50, 100]
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.ravel()
    
    x = np.linspace(0, 1, 200)
    
    for snap_idx, snap_round in enumerate(snapshots):
        algo.reset()
        
        # Run simulation until snapshot
        for t in range(snap_round):
            arm = algo.select_arm()
            reward = env.pull_arm(arm)
            algo.update(arm, reward)
        
        # Plot Beta distributions
        ax = axes[snap_idx]
        
        for arm in range(5):
            alpha, beta = algo.alpha[arm], algo.beta[arm]
            y = stats.beta.pdf(x, alpha, beta)
            
            label = f'Arm {arm} (α={alpha:.1f}, β={beta:.1f})'
            ax.plot(x, y, label=label, linewidth=2)
            
            # Mark true CTR
            true_ctr = env.true_ctrs[arm]
            ax.axvline(true_ctr, linestyle='--', alpha=0.5, linewidth=1)
        
        ax.set_xlabel('CTR', fontsize=11)
        ax.set_ylabel('Probability Density', fontsize=11)
        ax.set_title(f'After {snap_round} rounds', fontsize=13)
        ax.legend(fontsize=8, loc='upper right')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

visualize_thompson_evolution()

## 5. Comparative Analysis

### 5.1 Head-to-Head Comparison

In [ ]:
# Comprehensive comparison across different time horizons

def comprehensive_comparison(time_horizons, n_simulations=100):
    """
    Compare all algorithms across different time horizons
    """
    results = []
    
    for T in time_horizons:
        print(f"\nTesting T={T} rounds...")
        
        env = NewsEnvironment(n_articles=10, seed=42)
        
        algorithms = {
            'Epsilon-Greedy': EpsilonGreedy(10, epsilon=0.1),
            'UCB': UCB(10, c=np.sqrt(2)),
            'Thompson Sampling': ThompsonSampling(10)
        }
        
        comparison = compare_algorithms(algorithms, env, T, n_simulations)
        comparison['Time Horizon'] = T
        
        results.append(comparison)
    
    return pd.concat(results, ignore_index=True)

# Run comprehensive comparison
time_horizons = [100, 500, 1000, 2000, 5000]
comparison_df = comprehensive_comparison(time_horizons, n_simulations=50)

print("\n" + "="*80)
print("COMPREHENSIVE COMPARISON RESULTS")
print("="*80)
print(comparison_df.to_string())

In [ ]:
# Visualize scaling behavior

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot regret vs time horizon
for algo_name in ['Epsilon-Greedy', 'UCB', 'Thompson Sampling']:
    algo_data = comparison_df[comparison_df['Algorithm'] == algo_name]
    
    axes[0].errorbar(
        algo_data['Time Horizon'],
        algo_data['Avg Final Regret'],
        yerr=algo_data['Std Final Regret'],
        marker='o',
        label=algo_name,
        capsize=4,
        linewidth=2
    )

axes[0].set_xlabel('Time Horizon (T)', fontsize=12)
axes[0].set_ylabel('Average Cumulative Regret', fontsize=12)
axes[0].set_title('Regret Growth over Time', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].set_xscale('log')
axes[0].set_yscale('log')

# Plot average CTR vs time horizon
for algo_name in ['Epsilon-Greedy', 'UCB', 'Thompson Sampling']:
    algo_data = comparison_df[comparison_df['Algorithm'] == algo_name]
    
    axes[1].plot(
        algo_data['Time Horizon'],
        algo_data['Avg CTR'],
        marker='o',
        label=algo_name,
        linewidth=2
    )

# Add optimal CTR line
env = NewsEnvironment(n_articles=10, seed=42)
axes[1].axhline(env.optimal_ctr, color='red', linestyle='--', 
               label='Optimal CTR', linewidth=2)

axes[1].set_xlabel('Time Horizon (T)', fontsize=12)
axes[1].set_ylabel('Average CTR', fontsize=12)
axes[1].set_title('CTR Convergence', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Statistical Analysis

### 6.1 Hypothesis Testing

In [ ]:
# Statistical significance testing

def run_significance_tests(n_rounds=1000, n_simulations=100):
    """
    Run paired t-tests to determine if differences are statistically significant
    """
    env = NewsEnvironment(n_articles=10, seed=42)
    
    # Collect results
    eg_regrets = []
    ucb_regrets = []
    ts_regrets = []
    
    for _ in range(n_simulations):
        # Epsilon-Greedy
        algo_eg = EpsilonGreedy(10, epsilon=0.1)
        result_eg = run_simulation(algo_eg, env, n_rounds)
        eg_regrets.append(result_eg['regret'][-1])
        
        # UCB
        algo_ucb = UCB(10, c=np.sqrt(2))
        result_ucb = run_simulation(algo_ucb, env, n_rounds)
        ucb_regrets.append(result_ucb['regret'][-1])
        
        # Thompson Sampling
        algo_ts = ThompsonSampling(10)
        result_ts = run_simulation(algo_ts, env, n_rounds)
        ts_regrets.append(result_ts['regret'][-1])
    
    # Perform t-tests
    print("\nPaired T-Tests (Cumulative Regret)")
    print("="*60)
    
    # Thompson vs Epsilon-Greedy
    t_stat, p_val = stats.ttest_rel(ts_regrets, eg_regrets)
    print(f"Thompson Sampling vs Epsilon-Greedy:")
    print(f"  t-statistic: {t_stat:.4f}")
    print(f"  p-value: {p_val:.6f}")
    print(f"  Significant at α=0.05: {'Yes' if p_val < 0.05 else 'No'}")
    print()
    
    # Thompson vs UCB
    t_stat, p_val = stats.ttest_rel(ts_regrets, ucb_regrets)
    print(f"Thompson Sampling vs UCB:")
    print(f"  t-statistic: {t_stat:.4f}")
    print(f"  p-value: {p_val:.6f}")
    print(f"  Significant at α=0.05: {'Yes' if p_val < 0.05 else 'No'}")
    print()
    
    # UCB vs Epsilon-Greedy
    t_stat, p_val = stats.ttest_rel(ucb_regrets, eg_regrets)
    print(f"UCB vs Epsilon-Greedy:")
    print(f"  t-statistic: {t_stat:.4f}")
    print(f"  p-value: {p_val:.6f}")
    print(f"  Significant at α=0.05: {'Yes' if p_val < 0.05 else 'No'}")
    
    return eg_regrets, ucb_regrets, ts_regrets

eg_regrets, ucb_regrets, ts_regrets = run_significance_tests()

In [ ]:
# Visualize distributions

fig, ax = plt.subplots(figsize=(10, 6))

data = [eg_regrets, ucb_regrets, ts_regrets]
labels = ['Epsilon-Greedy', 'UCB', 'Thompson Sampling']

bp = ax.boxplot(data, labels=labels, patch_artist=True,
                notch=True, showmeans=True)

# Color boxes
colors = ['lightblue', 'lightgreen', 'lightcoral']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

ax.set_ylabel('Cumulative Regret', fontsize=12)
ax.set_title('Distribution of Cumulative Regret across 100 Simulations', 
            fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 7. Conclusions

### Key Findings:

1. **Thompson Sampling** typically achieves the **lowest regret** in practice
   - Naturally balances exploration and exploitation
   - Adapts uncertainty estimates over time
   - Often statistically significantly better than alternatives

2. **UCB** provides **strong theoretical guarantees**
   - Logarithmic regret bound
   - Deterministic (no randomness)
   - Performance depends on choosing appropriate $c$ parameter

3. **Epsilon-Greedy** is **simple but less efficient**
   - Linear regret with constant epsilon
   - Can achieve sublinear regret with decaying epsilon
   - Good baseline, easy to understand

### Practical Recommendations:

- **Use Thompson Sampling** for best empirical performance
- **Use UCB** when theoretical guarantees are important
- **Use Epsilon-Greedy** for quick prototypes or when simplicity matters

### Future Directions:

1. **Contextual bandits** with user/article features
2. **Non-stationary** environments (CTRs change over time)
3. **Batch updates** for computational efficiency
4. **Delayed feedback** in real systems
5. **Fairness constraints** in recommendations

---

**References**:

1. Auer, P., Cesa-Bianchi, N., & Fischer, P. (2002). Finite-time analysis of the multiarmed bandit problem.
2. Agrawal, S., & Goyal, N. (2012). Analysis of Thompson sampling for the multi-armed bandit problem.
3. Chapelle, O., & Li, L. (2011). An empirical evaluation of Thompson sampling.
4. Lattimore, T., & Szepesvári, C. (2020). Bandit Algorithms.